In [1]:
# import library
import pandas as pd
from scipy.sparse import load_npz
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb

In [2]:
#import data after preprocessing
train_df = pd.read_csv("./project_data_files/book_rating_train.csv")
test_df = pd.read_csv("./project_data_files/book_rating_test.csv")
X_train = load_npz('./x_train_combine_419_mi.npz')
X_test = load_npz('./x_test_419_mi.npz')
y_train = train_df['rating_label']

In [3]:
# split train and testing dataset
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print(y_val)

247      4.0
12038    3.0
1580     4.0
3128     4.0
20531    4.0
        ... 
12586    3.0
7683     4.0
12912    4.0
9134     4.0
7893     4.0
Name: rating_label, Length: 4613, dtype: float64


In [4]:
def output_predictions(prediction):
    #output result
    output_df = pd.DataFrame({'rating_label': prediction})
    output_df.index += 1
    output_df.index.name = 'id'
    output_df.to_csv('./predictions.csv')
    return

In [5]:
#try XGboost
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

dtrain = xgb.DMatrix(X_train, label=y_train-3)

params = {
    'max_depth': list(range(50, 80, 10)),
    'min_child_weight': list(range(1, 6, 2)),
}

gsearch = GridSearchCV(estimator=xgb.XGBClassifier(learning_rate = 0.2, objective='multi:softmax', subsample = 0.8, colsample_bytree=0.8, 
                                                   random_state = 42, num_class = 3), param_grid=params, n_jobs=4, cv=3)
gsearch.fit(X_train, y_train-3)

In [5]:
print( gsearch.best_params_, gsearch.best_score_)

{'max_depth': 10, 'min_child_weight': 3} 0.7093613098067149


In [13]:
params = {
    'gamma': [i/10 for i in range(0, 5)]
}
gsearch = GridSearchCV(estimator=xgb.XGBClassifier(learning_rate = 0.2, objective='multi:softmax', subsample = 0.8, colsample_bytree=0.8, 
                                                   random_state = 42, num_class = 3, alpha = 3, max_depth = 10, min_child_weight = 3), param_grid=params, cv=3)
gsearch.fit(X_train, y_train-3)

GridSearchCV(cv=3,
             estimator=XGBClassifier(alpha=3, base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.03, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=10,
                                  

In [14]:
print( gsearch.best_params_, gsearch.best_score_)

{'gamma': 0.3} 0.7088409620397353


In [ ]:
params = {
    'subsample': list(range(5, 10)*0.1),
    'colsample_bytree': list(range(5, 10)*0.1)
}

gsearch = GridSearchCV(estimator=xgb.XGBClassifier(learning_rate = 0.2, objective='multi:softmax',gamma = 0.3,
                                                   random_state = 42, num_class = 3, alpha = 3, max_depth = 10, min_child_weight = 3), param_grid=params, cv=3)

In [ ]:
param = {
    'learning_rate': list(range(1,10)*0.001)
}
gsearch = GridSearchCV(estimator=xgb.XGBClassifier(objective='multi:softmax', subsample = 0.8, colsample_bytree=0.8, 
                                                   random_state = 42, num_class = 3, alpha = 3, max_depth = 10, min_child_weight = 3), param_grid=params, cv=3)


In [39]:
dtest = xgb.DMatrix(X_val)
y_pred = model.predict(dtest)
y_pred = [round(value) for value in y_pred]
accuracy = accuracy_score(y_val-3, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 71.97%


In [27]:
dtest = xgb.DMatrix(X_test)
prediction = model.predict(dtest)
print(prediction+3)
output_predictions(prediction+3)

[4. 4. 4. ... 4. 4. 4.]
